In [1]:
import wntr  
import numpy as np 
import pandas as pd
import re

In [15]:
# Replace with appropriate path and filename
directory='../Network-Files/Network 3/'
filename='Network3_12hr_PDA.inp'
name_only=filename[0:-12]
print("Selected File: ",name_only)
abs_path=directory+filename

Selected File:  Network3_


In [47]:
demand_nodes=[]       # For storing list of nodes that have non-zero demands
desired_demands=[]    # For storing demand rates desired by each node for desired volume calculations
elevations=[]

# Creates a network model object using EPANET .inp file
network=wntr.network.WaterNetworkModel(abs_path)

# Iterates over the junction list in the Network object
for node in network.junctions():
    # Record node ID (name), desired demand (base_demand) in CMS, elevations, x and y coordinates
    demand_nodes.append(node[1].name)
    desired_demands.append(node[1].base_demand)
    elevations.append(node[1].elevation)



# Get the supply duration in minutes (/60) as an integer
supply_duration=int(network.options.time.duration/60)

In [48]:
new_duration_hr=24
new_duration_min=0
new_duration=new_duration_hr*60+new_duration_min
demand_multiplier=supply_duration/new_duration
if new_duration_min <10:
    new_duration_min="0"+str(new_duration_min)
else: new_duration_min=str(new_duration_min)

In [49]:
desired_demands=[demand*demand_multiplier*1000 for demand in desired_demands]
patterns=["       " for demand in desired_demands]
semicolons=[";" for demand in desired_demands]
node_section=pd.DataFrame(list(zip(demand_nodes,elevations,desired_demands,patterns,semicolons)))
node_section=node_section.to_string(header=False,index=False,col_space=10).splitlines()


In [50]:
# opens .inp file to read
file=open(abs_path,'r')
lines=[]            # list to store all lines in the .inp file
linecount=0         # Counter for the number of lines
junctions_marker=0  # To store the line number at which the junctions section starts
supply_duration_line=0

# Loops over each line in the input file 
for line in file:
    # Record the position of the phrase [JUNCTIONS] and add 2 to skip the header line
    if re.search('\[JUNCTIONS\]',line):
        junctions_marker=linecount+2
    if re.search('Duration',line):
        supply_duration_line=linecount
    linecount+=1
    # Store all lines in a list
    lines.append(line)
file.close()



# Inserts the created sections in their appropriate location in the list of lines
lines[supply_duration_line]="Duration      "+str(new_duration_hr)+":"+new_duration_min+"\n"
lines[junctions_marker:junctions_marker+len(node_section)]=node_section

print(lines[supply_duration_line])
# Opens a new file in the same directory to write the modified network .inp file in
file=open(directory+name_only+str(new_duration_hr)+"hr"+'_PDA.inp','w')
c=0     #line counter

# All lines added by this script are missing a new line character at the end, the conditional statements below add the new line character for these lines only and writes all lines to the file
for line in lines:
    if c>=junctions_marker and c<=junctions_marker+len(node_section)+len(node_section):
        file.write(line+'\n')
    else: file.write(line)    
    c+=1
file.close()

Duration      24:00

